In [ ]:
from typing import List
import xmlrpc.client
from bs4 import BeautifulSoup
import requests
from pathlib import Path


class RPCSender(object):
    def __init__(self, server_url: str, token: str):
        self.server_proxy = xmlrpc.client.ServerProxy(server_url)
        self.token = token

    def download_url(self, url: str, dir: str):
        options = {"dir": dir, "out": Path(url).name}
        gid = self.server_proxy.aria2.addUri("token:" + self.token, [url], options)
        print(f"Download started with GID: {gid}")

    def download_urls(self, urls: List[str], dir: str):
        for url in urls:
            self.download_url(url, dir)


def get_hub(url):
    url_cut = url.split('https://huggingface.co')[-1].replace('tree', 'resolve')
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    file_links = []

    for link in soup.find_all("a"):
        href = link.get("href")
        if href.startswith(url_cut):
            file_links.append(f"https://huggingface.co{href}")
    return file_links

In [ ]:
server_url = "http://192.155.1.11:6800/jsonrpc"
token = '113113'
model_url = "https://huggingface.co/decapoda-research/llama-7b-hf/tree/main"
model_download_dir = "/downloads/llama-7b-hf"
download_links = get_hub(model_url)
download_links

In [ ]:
aria2.download_urls(download_links, model_download_dir)